# 0. Импорт библиотек

In [97]:
import pandas as pd
import numpy as np

# 1. Знакомство с данными

### Справочник категорий

In [98]:
# Чтение данных
dim_category = pd.read_csv('data/dim_category.csv')
display(dim_category.head(2))

,КодКатегории,Уровень4
0,EM85913,Стеклоочиститель портативный
1,AM24371,Пылесос строительный


In [99]:
dim_category.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 718 entries, 0 to 717
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   КодКатегории  718 non-null    object
 1   Уровень4      718 non-null    object
dtypes: object(2)
memory usage: 11.3+ KB


In [100]:
# Описательная статистика
dim_category.describe()

,КодКатегории,Уровень4
count,718,718
unique,718,718
top,EM85913,Стеклоочиститель портативный
freq,1,1


В справочнике всего 718 уникальных категорий, пропусков нет.

In [101]:
# dim_category = dim_category.astype('category')

### Приоритеты характеристик

In [102]:
# Чтение данных
site_priorities = pd.read_excel('data/site_priorities.xlsx')
site_priorities.head(2)

,Категория,ID характеристики,Приоритет
0,Электронные книги,1000,720
1,Электронные книги,989,900


In [103]:
site_priorities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27296 entries, 0 to 27295
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Категория          27296 non-null  object
 1   ID характеристики  27296 non-null  int64 
 2   Приоритет          27296 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 639.9+ KB


In [104]:
# Описательная статистика
display(site_priorities['Приоритет'].describe())
site_priorities.nunique()

count    27296.000000
mean       825.076568
std         85.131890
min       -140.000000
25%        800.000000
50%        850.000000
75%        880.000000
max        900.000000
Name: Приоритет, dtype: float64

Категория             2654
ID характеристики    26610
Приоритет              105
dtype: int64

1. В справочнике приоритетов 2654 категории, 26610 характеристик, пропусков нет.
2. Диапазон приоритетов от -140 до 900

In [105]:
# Меняю тип данных на строковый
site_priorities['ID характеристики'] = site_priorities['ID характеристики'].astype('str')

### Справочник характеристик товара

In [106]:
# Чтение данных
product_specifications = pd.read_csv('data/product_specifications.csv')
product_specifications.head(2)

,Код,ВидХарактеристики,КодВидаХарактеристики,КодХарактеристики,Характеристика,Значимость
0,5334213,Наличие фильтра от мелких частиц,13494,163267,есть,200
1,5334213,Напряжение питания,1205937,60932058,220-240 В / 50 Гц,110


In [108]:
product_specifications = product_specifications.rename(columns={'Код':'КодТовара'})
product_specifications.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82949 entries, 0 to 82948
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   КодТовара              82949 non-null  int64 
 1   ВидХарактеристики      82949 non-null  object
 2   КодВидаХарактеристики  82949 non-null  int64 
 3   КодХарактеристики      82949 non-null  int64 
 4   Характеристика         82942 non-null  object
 5   Значимость             82949 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 3.8+ MB


In [109]:
# Проверка диапазона значимости
product_specifications['Значимость'].describe()

count    82949.000000
mean       199.946027
std         54.466492
min          0.000000
25%        190.000000
50%        218.000000
75%        240.000000
max        255.000000
Name: Значимость, dtype: float64

1. Диапазон "Значимость" от 0 до 250.
2. В таблице есть пропуски, их заполнением займемся далее.
3. Нужно преобразование типов данных.

In [112]:
product_specifications[['КодТовара', 'КодВидаХарактеристики', 'КодХарактеристики']] = product_specifications[['КодТовара', 'КодВидаХарактеристики', 'КодХарактеристики']].astype('str')

### Справочник Товар - Категория

In [113]:
# Чтение данных
dim_product = pd.read_excel('data/dim_product.xlsx')
dim_product.head(2)

,КодТовара,КодКатегории
0,142961,AM19415
1,142962,AM19415


In [114]:
dim_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6565 entries, 0 to 6564
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   КодТовара     6565 non-null   int64 
 1   КодКатегории  6565 non-null   object
dtypes: int64(1), object(1)
memory usage: 102.7+ KB


# 2. Обработка пропусков

In [115]:
product_specifications.isna().sum()

КодТовара                0
ВидХарактеристики        0
КодВидаХарактеристики    0
КодХарактеристики        0
Характеристика           7
Значимость               0
dtype: int64

<!-- Поскольку  -->

Поскольку сегментация должна происходить по характеристикам, то строки с незаполненными характеристиками нам не интересны, их можно удалить.

In [116]:
product_specifications.dropna(inplace=True)
print('Осталось пропусков ', product_specifications.isna().sum().sum())

Осталось пропусков  0


# 3. Базовая модель (baseline)

## Обработка признаков для модели